In [6]:
%load_ext autoreload
%autoreload 2

In [10]:
import Geneticos as gen
import Graficos as grf
import numpy as np
import pandas as pd
import itertools
import plotly
import optuna
import pickle
from Hyperparameters import GeneticAlgorithmOptimizer
from scipy.spatial.distance import pdist, squareform
from optuna.visualization import plot_optimization_history, plot_param_importances

### DEFINIENDO PARAMETROS DEL PROBLEMA

In [8]:
cantidad_ciudades = 100
gen.coordenadas_ciudades = [[0.21954616, 0.14581329],
                            [0.17421578, 0.18566283],
                            [0.29122291, 0.80457042],
                            [0.18040537, 0.80267858],
                            [0.3412426, 0.41644281],
                            [0.58179122, 0.49474069],
                            [0.67356906, 0.75094227],
                            [0.88793601, 0.47142495],
                            [0.00779237, 0.53205174],
                            [0.36181121, 0.58683286],
                            [0.01003692, 0.85785187],
                            [0.67929191, 0.24353308],
                            [0.6065664, 0.70519311],
                            [0.72948251, 0.43073781],
                            [0.9607949, 0.54489034],
                            [0.88817971, 0.07955406],
                            [0.52557476, 0.4696241],
                            [0.94650895, 0.03802156],
                            [0.64501347, 0.77524301],
                            [0.87601183, 0.78304364],
                            [0.36018579, 0.93557705],
                            [0.89366112, 0.60359089],
                            [0.97601195, 0.85817543],
                            [0.26127822, 0.71608585],
                            [0.90879018, 0.58434009],
                            [0.91013166, 0.12334488],
                            [0.91736713, 0.23586366],
                            [0.13667125, 0.00201317],
                            [0.94980787, 0.4336497],
                            [0.2038939, 0.93308361],
                            [0.83826848, 0.41281027],
                            [0.32066564, 0.49212394],
                            [0.14341033, 0.91744724],
                            [0.69439039, 0.58776669],
                            [0.7831435, 0.05133785],
                            [0.99532546, 0.77610824],
                            [0.48295153, 0.57015342],
                            [0.93819732, 0.79385582],
                            [0.29426788, 0.64406639],
                            [0.26814418, 0.23473205],
                            [0.16670826, 0.685144],
                            [0.60190034, 0.27357298],
                            [0.65943488, 0.21823561],
                            [0.09769489, 0.8826959],
                            [0.66464977, 0.86631929],
                            [0.3814482, 0.94021531],
                            [0.09677459, 0.10593996],
                            [0.17864128, 0.62865786],
                            [0.01510431, 0.44956381],
                            [0.23513167, 0.41254368],
                            [0.4549013, 0.01794302],
                            [0.80061994, 0.39180504],
                            [0.3417234, 0.37118961],
                            [0.19172995, 0.43508706],
                            [0.13512255, 0.99817623],
                            [0.11329438, 0.90948516],
                            [0.09481105, 0.52652043],
                            [0.60478417, 0.57487087],
                            [0.42949743, 0.35347706],
                            [0.69048789, 0.1950799],
                            [0.99041799, 0.45246021],
                            [0.58163492, 0.80503285],
                            [0.75869429, 0.66096662],
                            [0.6583406, 0.87208508],
                            [0.60404149, 0.32236654],
                            [0.63871647, 0.03338515],
                            [0.64207654, 0.20223676],
                            [0.12276697, 0.68577585],
                            [0.10845472, 0.84312461],
                            [0.53897218, 0.30451347],
                            [0.6133547, 0.02200327],
                            [0.0488243, 0.28018404],
                            [0.57981611, 0.23698127],
                            [0.3168622, 0.5841623],
                            [0.622078, 0.34672988],
                            [0.26934929, 0.63440813],
                            [0.90313144, 0.65774199],
                            [0.12093674, 0.75562179],
                            [0.08036233, 0.44383751],
                            [0.10992283, 0.43391352],
                            [0.08494473, 0.80382135],
                            [0.49516912, 0.774868],
                            [0.95936749, 0.2579975],
                            [0.93321228, 0.88949077],
                            [0.65689517, 0.02874112],
                            [0.19149822, 0.53687667],
                            [0.95414153, 0.91035188],
                            [0.50068953, 0.03360075],
                            [0.65037733, 0.63369655],
                            [0.18609162, 0.74971852],
                            [0.99697744, 0.22788147],
                            [0.73286254, 0.02269668],
                            [0.13865304, 0.58424392],
                            [0.26217996, 0.45843858],
                            [0.82195241, 0.4835036],
                            [0.77293438, 0.88474862],
                            [0.95244401, 0.80528087],
                            [0.35954457, 0.03645583],
                            [0.71485602, 0.81287991],
                            [0.06654975, 0.81823839]]
gen.matriz_tsp_distancias = squareform(pdist(gen.coordenadas_ciudades, 'euclidean'))

In [9]:
'''
Experimento 0:
Para un mismo conjunto de 100 ciudades se va a buscar los hyperparametros optimos: 
- cantidad_poblacion: trial.suggest_int(40, 80)
- elite_size: trial.suggest_int(5, 30)
- mutation_rate: trial.suggest_float(0.01, 0.31, step=0.05)
'''
# SELECCIONAR HYPERPARAMETROS OPTIMOS
cantidad_generaciones = 5000

# Creamos un optimizador con Optuna para correr pruebas en paralelo
optimizer = GeneticAlgorithmOptimizer(cantidad_generaciones, cantidad_ciudades)
study = optuna.create_study(direction='minimize')
study.optimize(optimizer.objective, n_trials=25, n_jobs=-1)

[I 2024-06-08 10:39:12,688] A new study created in memory with name: no-name-a29de388-cf58-4574-8a37-baa4956d33d1


Generation 0 | Best distance: 47.27647335331075
Generation 0 | Best distance: 45.330677512402325
Generation 0 | Best distance: 48.747460801132526
Generation 0 | Best distance: 47.34724437804884
Generation 0 | Best distance: 47.89122653089209
Generation 0 | Best distance: 46.02262987043481
Generation 0 | Best distance: 47.69554289032097
Generation 0 | Best distance: 46.55736470976538
Generation 0 | Best distance: 48.20726506383873
Generation 0 | Best distance: 48.106550363905804
Generation 0 | Best distance: 48.88131954181969
Generation 0 | Best distance: 49.42524682055848
Generation 0 | Best distance: 48.336873912946125
Generation 0 | Best distance: 48.30722694869717
Generation 0 | Best distance: 48.97442887291188
Generation 0 | Best distance: 44.93843785392909
Generation 0 | Best distance: 48.712018470852364
Generation 0 | Best distance: 45.078256403095494
Generation 0 | Best distance: 47.97950603209865
Generation 0 | Best distance: 46.38389068496826
Generation 0 | Best distance: 46.6

[I 2024-06-08 10:56:58,533] Trial 12 finished with value: 10.878849161499655 and parameters: {'cantidad_poblacion': 46, 'elite_size': 22, 'mutation_rate': 0.11}. Best is trial 12 with value: 10.878849161499655.


Final best distance: 10.878849161499655
Generation 0 | Best distance: 46.599115562026306
Generation 3000 | Best distance: 12.246429837515688
Generation 4200 | Best distance: 10.23299100024353
Generation 3700 | Best distance: 12.13550966859749
Generation 2800 | Best distance: 14.279454959885834
Generation 2800 | Best distance: 11.604200680122895
Generation 3200 | Best distance: 10.934448312646463
Generation 3400 | Best distance: 11.503842025929147
Generation 2900 | Best distance: 11.770720654715744
Generation 4300 | Best distance: 10.623501559396217
Generation 3900 | Best distance: 11.544650978129107
Generation 3700 | Best distance: 11.202057878888592
Generation 4800 | Best distance: 11.367723157101672
Generation 3600 | Best distance: 10.608376726585218
Generation 2700 | Best distance: 10.966013264785499
Generation 100 | Best distance: 26.97822210947609
Generation 4300 | Best distance: 10.230550399139773
Generation 2200 | Best distance: 12.169345116417546
Generation 2800 | Best distance

[I 2024-06-08 10:57:58,814] Trial 9 finished with value: 11.363968694036414 and parameters: {'cantidad_poblacion': 47, 'elite_size': 20, 'mutation_rate': 0.01}. Best is trial 12 with value: 10.878849161499655.


...

Final best distance: 10.902462812389004
Generation 4500 | Best distance: 10.705553333418889
Generation 4600 | Best distance: 10.67228500240286


[I 2024-06-08 11:10:10,830] Trial 4 finished with value: 9.795476368578194 and parameters: {'cantidad_poblacion': 64, 'elite_size': 15, 'mutation_rate': 0.26}. Best is trial 8 with value: 9.25048903760565.


Generation 4700 | Best distance: 10.488357585449824
Final best distance: 9.795476368578194
Generation 4800 | Best distance: 11.061184773464264
Generation 4800 | Best distance: 10.488357585449824Generation 4900 | Best distance: 11.027420042954605



[I 2024-06-08 11:10:16,226] Trial 10 finished with value: 11.027420042954605 and parameters: {'cantidad_poblacion': 64, 'elite_size': 15, 'mutation_rate': 0.31}. Best is trial 8 with value: 9.25048903760565.
[I 2024-06-08 11:10:16,294] Trial 3 finished with value: 10.488357585449824 and parameters: {'cantidad_poblacion': 64, 'elite_size': 8, 'mutation_rate': 0.26}. Best is trial 8 with value: 9.25048903760565.


Generation 4900 | Best distance: 10.488357585449824
Final best distance: 11.027420042954605
Final best distance: 10.488357585449824


In [11]:
# Open the file in binary read mode
with open('study_25.pkl', 'rb') as file: # study_10.pkl
    # Deserialize the object from the file
    study = pickle.load(file)
    
# # Open a file in binary write mode
# with open('study_25.pkl', 'wb') as file:
#     # Serialize and write the object to the file
#     pickle.dump(study, file)

In [12]:
# Obtenemos los mejores hyperametros
print(f"Best hyperparameters: {study.best_params}")
print(f"Best objective value: {study.best_value}")

Best hyperparameters: {'cantidad_poblacion': 79, 'elite_size': 27, 'mutation_rate': 0.060000000000000005}
Best objective value: 9.25048903760565


In [14]:
# Mostramos las gracias de optimizacion e imprtancia
# optuna_history_fig = plot_optimization_history(study)
optuna_importances_fig = plot_param_importances(study)
# optuna_history_fig.show()
optuna_importances_fig.show()